# MapsTo in pySBOL2

The `MapsTo` class in the Synthetic Biology Open Language (SBOL) is used to explicitly state that two `ComponentInstance` objects, often from different levels of design hierarchy, represent the same biological entity. It is most often used when `ModuleDefinition` and `ComponentDefinition` objects are composed using `Module` and `ComponentInstance` objects.

`MapsTo` objects define how a `ComponentInstance` in a higher-level design relates to a `ComponentInstance` in a lower-level design through identity and refinement relationships.

`MapsTo` objects have the following required properties:

-   `local`: Refers to the `ComponentInstance` in the higher-level design.
-   `remote`: Refers to the `ComponentInstance` in the lower-level design. The referenced instance must have `access="public"`.
-   `refinement`: Specifies how to interpret the relationship between the local and remote instances using a URI. For example: `http://sbols.org/v2#useRemote`.

This example demonstrates linking a `FunctionalComponent` in a high-level toggle switch module to one in a lower-level LacI inverter using a `MapsTo` object. We will:

1. Define two `ModuleDefinition` objects (toggle switch and LacI inverter).
2. Add `FunctionalComponent` instances to both modules.
3. Create a `Module` to instantiate the LacI inverter inside the toggle switch.
4. Add a `MapsTo` to specify how the high-level LacI relates to the inverter's TF.

For more information on the `MapsTo` class and its properties, refer to page 30 in the SBOL 2.x.x specification document.

In [1]:
import sbol2

In [2]:
doc = sbol2.Document()

# Set a namespace for the document
sbol2.setHomespace('https://github.com/SynBioDex/SBOL-Notebooks')

Creating a MapsTo Object

In [3]:
# Create lower-level ModuleDefinition
inverter_md = sbol2.ModuleDefinition('laci_inverter')
tf_fc = inverter_md.functionalComponents.create('TF')
tf_fc.definition = tf_fc
tf_fc.access = sbol2.SBOL_ACCESS_PUBLIC
tf_fc.direction = sbol2.SBOL_DIRECTION_IN_OUT

# Create higher-level ModuleDefinition
toggle_md = sbol2.ModuleDefinition('toggle_switch')
laci_fc = toggle_md.functionalComponents.create('LacI')
laci_fc.definition = laci_fc
laci_fc.access = sbol2.SBOL_ACCESS_PRIVATE
laci_fc.direction = sbol2.SBOL_DIRECTION_IN_OUT

# Create a Module instance to include the inverter
mod = toggle_md.modules.create('laci_inverter_instance')
mod.definition = inverter_md.identity

# Create the MapsTo
mapping = mod.mapsTos.create('LacI_mapping')
mapping.local = laci_fc.identity
mapping.remote = tf_fc.identity
mapping.refinement = sbol2.SBOL_REFINEMENT_USE_REMOTE

# Add ModuleDefinitions to the document
doc.addModuleDefinition(inverter_md)
doc.addModuleDefinition(toggle_md)

In [4]:
# Validate the document
report = doc.validate()

if report == "Valid.":
    doc.write("example_mapsto.xml")
else:
    print(report)